In [ ]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,capacities,lambdas
import numpy as np
from util import welfare_metrics,student_by_col,bayes_update_grade,grades_gr_to_grades_col
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est,anal_cond_exp
from scipy.optimize import fsolve
from scipy.stats import norm, multivariate_normal
from param_def import chi,sigma,lambdas

In [ ]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [ ]:
def cdf(x, sigma):
    return norm.cdf(x, scale = sigma)

In [ ]:
def cdfm(x, y, sigma, rho):
    return multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [ ]:
def cdfmsi(x, y, sigma, rho):
    return norm.cdf(y, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [ ]:
def cdfmis(x, y, sigma, rho):
    return norm.cdf(x, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [ ]:
df1 = pd.DataFrame({'A':updated_grade_estimated[0][0],'B':updated_grade_estimated[1][0],'pref':np.array(stud_pref[0]).T[0]})
df2 = pd.DataFrame({'A':updated_grade_estimated[0][1],'B':updated_grade_estimated[1][1],'pref':np.array(stud_pref[1]).T[0]})

**Check if empirical count is equal to ECDF and EPDF (this should be equal)**

In [ ]:
gr1_ecdf_pb,gr2_ecdf_pb,multi_ecdf_gr1,multi_ecdf_gr2= sampling_ecdf(grade_estimated,stud_pref,new_cutoff_values[0],new_cutoff_values[1],chi,sigma,lambdas ,type = 'right_partial')

$P_1(W_A > P_A),P_2(W_A > P_A)$

In [ ]:
#Empirical counting of gr1 and gr2
print('Group 1:', len(df1.loc[df1['A']>new_cutoff_values[0]])/len(df1))
print('Group 2:', len(df2.loc[df2['A']>new_cutoff_values[0]])/len(df2))

Group 1: 0.2865
Group 2: 0.28941666666666666


In [ ]:
#Theoretical value of gr1 and gr 2
print('Group 1:', 1 - cdf(new_cutoff_values[0],sigma_i))
print('Group 2:', 1 - cdf(new_cutoff_values[0],sigma_ii))


Group 1: 0.2840824007231191
Group 2: 0.2840824007231191


It's not that much difference, this difference is acceptable.

$P_1(W_B > P_B), P_2(W_B >P_B)$.  Here we update part of $W_B$ based on student preference.

In [ ]:
print('Group 1:', 1 - gr1_ecdf_pb)
print('Group 2:', 1 - gr2_ecdf_pb)

Group 1: 0.261625
Group 2: 0.2523333333333333


In [ ]:
print('Group 1:', len(df1.loc[df1['B']>new_cutoff_values[1]])/len(df1))
print('Group 2:', len(df2.loc[df2['B']>new_cutoff_values[1]])/len(df2))

Group 1: 0.261625
Group 2: 0.25233333333333335


The result is not really surprised considering the fact that the CDF in the value is basically empirical base on the same data as the data being used in the empirical counting.

$P_1(W_A>P_A,W_B<P_B),P_2(W_A>P_A,W_B<P_B)$

In [ ]:
#Theoretical 
print('Group 1:', gr1_ecdf_pb - multi_ecdf_gr1)
print('Group 2:', gr2_ecdf_pb - multi_ecdf_gr2)

Group 1: 0.1481987720284964
Group 2: 0.14113387773241126


In [ ]:
#Empirical counting
print('Group 1:', len(df1.loc[(df1['B'] < new_cutoff_values[1])&(df1['A'] > new_cutoff_values[0])])/len(df1))
print('Group 2:', len(df2.loc[(df2['B'] < new_cutoff_values[1])&(df2['A'] > new_cutoff_values[0])])/len(df2))

Group 1: 0.148125
Group 2: 0.14108333333333334


The results are closed enough

$P_1(W_A < P_A,W_B > P_B),P_2(W_A < P_A, W_B > P_B)$

In [ ]:
#Theoretical
print('Group 1:', cdf(new_cutoff_values[0],sigma_i) - multi_ecdf_gr1)
print('Group 2:', cdf(new_cutoff_values[0],sigma_ii) - multi_ecdf_gr2)

Group 1: 0.12574137130537733
Group 2: 0.10938481034262548


In [ ]:
#Empirical counting
print('Group 1:', len(df1.loc[(df1['B'] > new_cutoff_values[1])&(df1['A'] < new_cutoff_values[0])])/len(df1))
print('Group 1:', len(df2.loc[(df2['B'] > new_cutoff_values[1])&(df2['A'] < new_cutoff_values[0])])/len(df2))

Group 1: 0.12325
Group 1: 0.104


The difference in the results are acceptable.

Earlier, we were only interested in the overall level without accounting for preference. Now we need to take the prefernce into consideration. We want to check whether or not the value $\gamma_j (\beta_j /1 - \beta_j)$ is equal to the empirical counting of the mass of all student belong to group $j$ and interested in $A (B)$. 

In [ ]:
#Theoretical
print('Group 1 interested in A:', prop_gp[0]*prop_all_g_prefer[0])
print('Group 1 interested in B:', prop_gp[0]*(1-prop_all_g_prefer[0]))
print('Group 2 interested in A:', prop_gp[1]*prop_all_g_prefer[1])
print('Group 2 interested in B:', prop_gp[1]*(1-prop_all_g_prefer[1]))

Group 1 interested in A: 0.16000000000000003
Group 1 interested in B: 0.24
Group 2 interested in A: 0.24
Group 2 interested in B: 0.36


In [ ]:
#Empirical
print('Group 1 interested in A:', len(df1.loc[df1['pref']==0])/20000)
print('Group 1 interested in B:', len(df1.loc[df1['pref']==1])/20000)
print('Group 2 interested in A:', len(df2.loc[df2['pref']==0])/20000)
print('Group 2 interested in B:', len(df2.loc[df2['pref']==1])/20000)

Group 1 interested in A: 0.16225
Group 1 interested in B: 0.23775
Group 2 interested in A: 0.2401
Group 2 interested in B: 0.3599


The difference in results are acceptable.

Now we want to check whether or not the theoretical value $\gamma_j * \beta_j * P_j(W_A > P_A)$,$\gamma_j * (1-\beta_j)*P_J(W_A > P_A,W_B < P_B)$ is equal to the empirical counting

In [ ]:
#Theoretical
print('Group 1 get first choice and pref A as proportion of total mass:', prop_gp[0]*prop_all_g_prefer[0]*(1- cdf(new_cutoff_values[0],sigma_i)))
print('Group 1 get first choice and pref A as absolute number of student:', prop_gp[0]*prop_all_g_prefer[0]*(1- cdf(new_cutoff_values[0],sigma_i))*20000)
print('Group 2 get first choice and pref A as proportion of total mass:', prop_gp[1]*prop_all_g_prefer[1]*(1- cdf(new_cutoff_values[0],sigma_ii)))
print('Group 2 get first choice and pref A as absolute number of student:', prop_gp[1]*prop_all_g_prefer[1]*(1- cdf(new_cutoff_values[0],sigma_ii))*20000)

Group 1 get first choice and pref A as proportion of total mass: 0.04545318411569906
Group 1 get first choice and pref A as absolute number of student: 909.0636823139812
Group 2 get first choice and pref A as proportion of total mass: 0.06817977617354858
Group 2 get first choice and pref A as absolute number of student: 1363.5955234709716


In [ ]:
#Empirical
print('Group 1 get first choice and pref A as proportion of total mass:', len(df1.loc[(df1['A'] > new_cutoff_values[0])&(df1['pref']==0)])/20000)
print('Group 1 get first choice and pref A as absolute number of student:', len(df1.loc[(df1['A'] > new_cutoff_values[0])&(df1['pref']==0)]))
print('Group 2 get first choice and pref A as proportion of total mass:', len(df2.loc[(df2['A'] > new_cutoff_values[0])&(df2['pref']==0)])/20000)
print('Group 2 get first choice and pref A as absolute number of student:', len(df2.loc[(df2['A'] > new_cutoff_values[0])&(df2['pref']==0)]))

Group 1 get first choice and pref A as proportion of total mass: 0.04815
Group 1 get first choice and pref A as absolute number of student: 963
Group 2 get first choice and pref A as proportion of total mass: 0.06785
Group 2 get first choice and pref A as absolute number of student: 1357


In [ ]:
#Theoretical
print('Group 1 get second choice and pref B as proportion of total mass:', prop_gp[0]*(1-prop_all_g_prefer[0])*(gr1_ecdf_pb- multi_ecdf_gr1))
print('Group 1 get second choice and pref B as absolute number of student:', prop_gp[0]*(1-prop_all_g_prefer[0])*(gr1_ecdf_pb- multi_ecdf_gr1)*20000)
print('Group 2 get second choice and pref B as proportion of total mass:', prop_gp[1]*(1-prop_all_g_prefer[1])*(gr2_ecdf_pb- multi_ecdf_gr2))
print('Group 2 get second choice and pref B as absolute number of student:', prop_gp[1]*(1-prop_all_g_prefer[1])*(gr2_ecdf_pb- multi_ecdf_gr2)*20000)

Group 1 get second choice and pref B as proportion of total mass: 0.035567705286839134
Group 1 get second choice and pref B as absolute number of student: 711.3541057367827
Group 2 get second choice and pref B as proportion of total mass: 0.05080819598366805
Group 2 get second choice and pref B as absolute number of student: 1016.163919673361


In [ ]:
#Empirical
print('Group 1 get second choice and pref B as proportion of total mass:', len(df1.loc[(df1['A'] > new_cutoff_values[0])&(df1['pref']==1)&(df1['B']<new_cutoff_values[1])])/20000)
print('Group 1 get second choice and pref B as absolute number of student:', len(df1.loc[(df1['A'] > new_cutoff_values[0])&(df1['pref']==1)&(df1['B']<new_cutoff_values[1])]))
print('Group 2 get second choice and pref B as proportion of total mass:', len(df2.loc[(df2['A'] > new_cutoff_values[0])&(df2['pref']==1)&(df2['B']<new_cutoff_values[1])])/20000)
print('Group 2 get second choice and pref B as absolute number of student:', len(df2.loc[(df2['A'] > new_cutoff_values[0])&(df2['pref']==1)&(df2['B']<new_cutoff_values[1])]))

Group 1 get second choice and pref B as proportion of total mass: 0.04305
Group 1 get second choice and pref B as absolute number of student: 861
Group 2 get second choice and pref B as proportion of total mass: 0.06535
Group 2 get second choice and pref B as absolute number of student: 1307


In [ ]:
from util import market_clear_noise_corr
Pa = new_cutoff_values[0]
Pb = new_cutoff_values[1]
prop = prop_gp[0]
prefi = prop_all_g_prefer[0]
prefii = prop_all_g_prefer[1]
capA = capacities_rate[0]
capB = capacities_rate[1]

In [ ]:
f1,f2 = market_clear_noise_corr(new_cutoff_values[0], new_cutoff_values[1], grade_estimated, stud_pref, prop_gp[0], capacities_rate[0], capacities_rate[1], prop_all_g_prefer[0], prop_all_g_prefer[1], sigma_i, sigma_ii, cor_i, cor_ii,chi,sigma,lambdas,bayes='right_partial')


In [ ]:
print(f1,f2)

8.861559754824722e-06 9.738910904666387e-07
